In [25]:
from tqdm import tqdm
import re
import pandas as pd
import os
from transliterate import translit
import os

Работаем с примерами из RLC.

In [8]:
def open_table(file):
    xl = pd.ExcelFile(file)
    df1 = xl.parse('Sheet1')
    df_rlc = df1.iloc [:, [0,3,4,5,6,8,9,10,11,12,13,14]] #Выбираем те столбцы, которые могут понадобиться.
    df_rlc = df_rlc.fillna('') #NaN заменяем '', чтобы не возникало ошибок при дальнейшей работе программы.
    return df_rlc

In [9]:
df_1_rlc = open_table('results_1_100.xlsx')

In [5]:
df_1_rlc

,Номер примера,Автор,Язык,Русский язык,Уровень,Оригинальное предложение,Исправленное предложение,Тег,Ошибка,Исправление,Разметчик,Комментарий
0,1,"Katya (F, HL, AM)",eng,HL,AM,Этот металл может сохраняться в организме чело...,Этот металл может сохраняться в организме чело...,Syntax,воздействую,воздействуя,Annotator1,
1,1,"Katya (F, HL, AM)",eng,HL,AM,Этот металл может сохраняться в организме чело...,Этот металл может сохраняться в организме чело...,"Ortho, Altern",желудожно-кишечный,желудочно-кишечный,MZambrzhitskaya,
2,2,"Katya (F, HL, AM)",eng,HL,AM,Основной путь попадания свинца в организм - {{...,Основной путь попадания свинца в организм - * ...,"Syntax, Insert, Transfer",это,,Annotator1,
3,3,"Olga (F, HL, AL)",eng,HL,AL,Стали строил монументальные здания чтобы эти з...,"*Сталин* строил монументальные *здания,* чтобы...",Ortho,Стали,Сталин,ubagryanskaya,
4,3,"Olga (F, HL, AL)",eng,HL,AL,Стали строил монументальные здания чтобы эти з...,"*Сталин* строил монументальные *здания,* чтобы...",Ref,эти здания,они,ubagryanskaya,
...,...,...,...,...,...,...,...,...,...,...,...,...
1027,425,"Anna (F, HL, AL)",eng,HL,AL,Дети привыкают к вкусу такой пищи и так же {{р...,Дети привыкают к вкусу такой пищи и так же рек...,syntax,рекламам,,admin,
1028,425,"Anna (F, HL, AL)",eng,HL,AL,Дети привыкают к вкусу такой пищи и так же {{р...,Дети привыкают к вкусу такой пищи и так же рек...,orpho,превычки,,admin,
1029,426,"Anna (F, HL, AL)",eng,HL,AL,"{{По}} {{статистике}}, {{если}} {{у}} {{одного...",По статистике если у одного родителя избыточны...,syntax,,у,admin,
1030,427,"Anna (F, HL, AL)",eng,HL,AL,{{между}} 16% и 33% детей и подростков имели и...,между 16% и 33% детей и подростков имели избыт...,syntax,между,,admin,


In [10]:
def to_str(df):
    str_for_dict = (df['Оригинальное предложение'] + ' | ' + df['Тег'] + ' | ' + df['Ошибка']+ ' | ' + df['Исправление']).to_list()
    return str_for_dict

In [11]:
list_1_rlc = to_str(df_1_rlc)

In [12]:
def dict_from_list(list_from_str):
    dict_df_rlc = {}

    for element in tqdm(list_from_str):
        element = element.split(' | ')
        sentence = element[0]
        sentence = sentence.replace('{{', '')
        sentence = sentence.replace('}}', '')
        if sentence[0] != '&':
            if sentence[0] == '.':
                sentence = sentence[1:] 
            if sentence[len(sentence)-1] != '.':
                sentence = sentence + '.'
            if sentence not in dict_df_rlc:
                dict_df_rlc[sentence] = dict({'Tag': [], 'Mistake': [], 'Correct': []})
                dict_df_rlc[sentence]['Tag'].append(element[1])
                dict_df_rlc[sentence]['Mistake'].append(element[2])
                dict_df_rlc[sentence]['Correct'].append(element[3])
    return dict_df_rlc

In [13]:
dict_df_1_rlc = dict_from_list(list_1_rlc)

100%|██████████████████████████████████████████████████████████████████████████| 1032/1032 [00:00<00:00, 344600.09it/s]


In [18]:
def to_file_for_udpipe(dict_rlc):
    with open('original_rlc.txt', 'a', encoding = 'utf-8') as f:
        for element in dict_rlc:
            f.write(element)
            f.write('\n')
    return f

In [19]:
to_file_for_udpipe(dict_df_1_rlc)

<_io.TextIOWrapper name='original_rlc.txt' mode='a' encoding='utf-8'>

Работаем с расшифровками.

In [2]:
def from_file(path_text):
    with open('all_texts.txt', 'a', encoding='utf-8') as f_main:
        for filename in os.listdir(path_text):
            with open(os.path.join(path_text, filename), 'r') as f:
                for line in f.readlines():
                    if '*CHI:' in line:
                        if '\\tab' in line:
                            line = line.split('\\tab')
                        elif '\t' in line:
                            line = line.split('\t')
                        line_new = line[1]
                        line_new = line_new[:-2]
                        f_main.write(line_new)
                        f_main.write('\n')
    return f_main

In [3]:
from_file('texts_transl')

<_io.TextIOWrapper name='all_texts.txt' mode='a' encoding='utf-8'>

In [12]:
def translit_to_rus(file):
    lines = []
    with open('all_texts.txt', 'r', encoding='utf-8') as f:
        with open('translit.txt', 'a', encoding='utf-8') as f2:  
            for line in f.readlines():
                ru_text = translit(line, 'ru')
                ru_text = ru_text.replace('@', '')
                ru_text = ru_text.replace('[', '')
                ru_text = ru_text.replace(']', '')
                ru_text = ru_text.replace('+/', '')
                ru_text = ru_text.replace('/', '')
                ru_text = ru_text.replace('#', '')
                ru_text = ru_text.replace('=', '')
                ru_text = ru_text.replace('&', '')
                ru_text = ru_text.replace('2', '')
                ru_text = ru_text.replace('0', '')
                ru_text = ru_text.replace('е~', 'э')
                ru_text = ru_text.replace('Е~', 'Э')
                if ru_text[-2] != '.' and ru_text[-2] != '?':
                    ru_text = ru_text.replace(ru_text[-1], '.\n')
                if len(ru_text.split()) > 2:
                    if 'xxx' not in ru_text:
                        if ru_text not in lines:
                            f2.write(ru_text)
                            lines.append(ru_text)
    return f2

In [13]:
translit_to_rus('all_texts.txt')

<_io.TextIOWrapper name='translit.txt' mode='a' encoding='utf-8'>

Расшифровки без транслитерации

In [93]:
def txt_rus(path_text):
    lines = []
    with open('txt_rus.txt', 'a', encoding='utf-8') as f_main:
        for filename in os.listdir(path_text):
            with open(os.path.join(path_text, filename), 'r', encoding='utf-16-le') as f:
                for line in f.readlines():
                    if '*CHI:' in line:
                        line = line.split('\t')
                        line_new = line[1]
                        line_new = line_new.replace('&', '')
                        line_new = line_new.replace('@', '')
                        line_new = line_new.replace('[', '')
                        line_new = line_new.replace(']', '')
                        line_new = line_new.replace('2', '')
                        line_new = line_new.replace('+/', '')
                        line_new = line_new.replace('#', '')
                        line_new = line_new.replace('(', '')
                        line_new = line_new.replace(')', '')
                        line_new = line_new.replace('2', '')
                        line_new = line_new.replace('/', '')
                        line_new = line_new.replace('errv-', '')
                        if len(line_new.split()) > 2:
                            if ('xxx' not in line_new) and ('ххх' not in line_new) and ('=' not in line_new):
                                if line_new not in lines:
                                    f_main.write(line_new)
                                    lines.append(line_new)
    return f_main

In [94]:
txt_rus('texts')

<_io.TextIOWrapper name='txt_rus.txt' mode='a' encoding='utf-8'>